# Setting up Variables and Functions

In [1]:
# Importing Pandas to create DataFrame
import pandas as pd
import numpy as np
import math

A_VALUE = 50 # remains the same
B_VALUE = 100 
C_VALUE = 300

A_VALUE_S = 50 # remains the same
B_VALUE_S = 0 
C_VALUE_S = 3000


BETA1 = 1
BETA2 = 0.01
BETA3 = 0.01
BETA4 = 0.01
BETA_S = 0.001

### Calculating X and Reputation Score for Buyers/Sellers

In [2]:
'''
Account Age Increment: in Months 
Time_btwn_transactions: in hours
price: in wei (min:100)
'''

def calculateX_Buyer(old_x, time_btwn_transactions, account_age):
    global BETA1, BETA2, BETA3
    beta1 = BETA1
    beta2 = BETA2
    beta3 = BETA3
    return old_x*beta1 + beta2*time_btwn_transactions + beta3*account_age

def calculateX_Seller(oldX, rep_score, rincoming, raverage):
    global BETA_S
    return oldX + rep_score * (rincoming-raverage)*BETA_S


# calculating reputation score 

def calculateRep_Buyer(x):
    global A_VALUE, B_VALUE, C_VALUE
    a = A_VALUE
    b = B_VALUE
    c = C_VALUE
    temp1 = x-b
    temp2 = c + (x-b)*(x-b)
    temp3 = temp1/math.sqrt(temp2)
    temp4 = temp3 +1
    return a * temp4


def calculateRep_Seller(x):
    global A_VALUE_S, B_VALUE_S, C_VALUE_S
    a = A_VALUE_S
    b = B_VALUE_S
    c = C_VALUE_S
    temp1 = x-b
    temp2 = c + (x-b)*(x-b)
    temp3 = temp1/math.sqrt(temp2)
    temp4 = temp3 +1
    return a * temp4
    

### Calculating X and Reputation Score for Sellers

In [80]:
AVG_REVIEW = 50 #start 
OLD_X = 0 #start
R_INCOMING = 60
REP_SCORE = 1
AVG_REVIEW = calculateRep_Seller(calculateX_Seller(OLD_X, REP_SCORE, R_INCOMING, AVG_REVIEW))
print(AVG_REVIEW)

50.0091287091396


# Simulation 1: Can a user artificially inflate his reputation score within a short period of time? 


**Considerations**
1. If so, what is the tolerance?
2. How long before the reputation can be increased?


In [3]:
#declaring first time user parameters
user_age = 0
user_total_price_reviews = 0
total_reviews = 0
user_x = 0
user_rep = 0

repList = []
xList = []
repList.append(0)
xList.append(0)

A_VALUE = 50 # remains the same
B_VALUE = 5 
C_VALUE = 10

while user_rep < 99:
    total_reviews += 1
    user_x = calculateX_Buyer(user_x, 1, 0)
    xList.append(user_x)
    user_rep = calculateRep_Buyer(user_x)
    repList.append(user_rep)

        
print("Number of transactions needed: ", len(repList)-1)
print("Number of hours: ",(len(repList)-1))
print("Number of days: ", (len(repList)-1)/24)


Number of transactions needed:  2058
Number of hours:  2058
Number of days:  85.75


However, the above values depend significantly on the hyperparameters:
1. B_VALUE
2. C_VALUE
3. BETA1
4. BETA2
5. BETA3
6. BETA4

In [24]:
# keeping Beta 1 to Beta 4 the same, and C_Value the same 
# we can see how the B_Value will change the effect
# the user spamming the system every hour

df = pd.DataFrame(columns = ['B_VALUE', 'C_VALUE', 'BETA1', 'BETA2', 'BETA3', 'BETA4', 'transactions', 'ETH', 'User_age', 'hours', 'days'])

def sim1(b, c, b1):
    global B_VALUE, C_VALUE, BETA1, BETA2, BETA3, BETA4
    B_VALUE = b
    C_VALUE = c
    BETA1 = b1
    user_age = 0
    total_time_passed = 0.0
    user_x = 0
    user_rep = 0
    repList = []
    xList = []
    while user_rep < 99:
        total_time_passed += 1.0
        user_age = total_time_passed/24/30 #divide by 24 hours, divide by 30days
        
        user_x = calculateX_Buyer(user_x, 1, int(user_age))
        xList.append(user_x)
        user_rep = calculateRep_Buyer(user_x)
        repList.append(user_rep)
        
    return len(repList), (len(repList))*2322070000000000/(10**18), total_time_passed, total_time_passed/24, user_age

#the user tries to inflate within a month and price 100wei, time between transactions is one hour
for b in range(5, 100, 5):
    for c in range(5, 100, 5):
        b1 = 1
        for b2 in [0.01, 0.1]:
            for b3 in [0.01, 0.1]:
                for b4 in [0.01, 0.1]:
                    transactions, ETH, hours, days, user_age = sim1(b, c, b1)
                    df.loc[len(df)] = [b, c, b1, b2, b3, b4, transactions, ETH, user_age, hours, days]
    
df

,B_VALUE,C_VALUE,BETA1,BETA2,BETA3,BETA4,transactions,ETH,User_age,hours,days
0,5.0,5.0,1.0,0.01,0.01,0.01,1161.0,2.695923,1.612500,1161.0,48.375000
1,5.0,5.0,1.0,0.01,0.01,0.10,1161.0,2.695923,1.612500,1161.0,48.375000
2,5.0,5.0,1.0,0.01,0.10,0.01,1161.0,2.695923,1.612500,1161.0,48.375000
3,5.0,5.0,1.0,0.01,0.10,0.10,1161.0,2.695923,1.612500,1161.0,48.375000
4,5.0,5.0,1.0,0.10,0.01,0.01,1161.0,2.695923,1.612500,1161.0,48.375000
...,...,...,...,...,...,...,...,...,...,...,...
2883,95.0,95.0,1.0,0.01,0.10,0.10,4183.0,9.713219,5.809722,4183.0,174.291667
2884,95.0,95.0,1.0,0.10,0.01,0.01,4183.0,9.713219,5.809722,4183.0,174.291667
2885,95.0,95.0,1.0,0.10,0.01,0.10,4183.0,9.713219,5.809722,4183.0,174.291667
2886,95.0,95.0,1.0,0.10,0.10,0.01,4183.0,9.713219,5.809722,4183.0,174.291667


In [25]:
df.to_csv('Simulation_1_1Hour.csv')

In [26]:
# keeping Beta 1 to Beta 4 the same, and C_Value the same 
# we can see how the B_Value will change the effect
# the user spamming the system every 24 hours

df = pd.DataFrame(columns = ['B_VALUE', 'C_VALUE', 'BETA1', 'BETA2', 'BETA3', 'BETA4', 'transactions', 'ETH', 'User_age','hours', 'days'])

def sim1(b, c, b1):
    global B_VALUE, C_VALUE, BETA1, BETA2, BETA3, BETA4
    B_VALUE = b
    C_VALUE = c
    BETA1 = b1
    user_age = 0
    total_time_passed = 0.0
    user_x = 0
    user_rep = 0
    repList = []
    xList = []
    while user_rep < 99:
        total_time_passed += 24.0
        user_age = total_time_passed/24/30 #divide by 24 hours, divide by 30days
        
        user_x = calculateX_Buyer(user_x, 24, int(user_age))
        xList.append(user_x)
        user_rep = calculateRep_Buyer(user_x)
        repList.append(user_rep)
        
    return len(repList), (len(repList))*2322070000000000/(10**18), total_time_passed, total_time_passed/24, user_age

for b in range(5, 200, 5):
    for c in range(5, 200, 5):
        b1 = 1
        for b2 in [0.01, 0.1]:
            for b3 in [0.01, 0.1]:
                for b4 in [0.01, 0.1]:
                    transactions, ETH, hours, days, user_age = sim1(b, c, b1)
                    df.loc[len(df)] = [b, c, b1, b2, b3, b4, transactions, ETH, user_age, hours, days]
    
df

,B_VALUE,C_VALUE,BETA1,BETA2,BETA3,BETA4,transactions,ETH,User_age,hours,days
0,5.0,5.0,1.0,0.01,0.01,0.01,65.0,0.150935,2.166667,1560.0,65.0
1,5.0,5.0,1.0,0.01,0.01,0.10,65.0,0.150935,2.166667,1560.0,65.0
2,5.0,5.0,1.0,0.01,0.10,0.01,65.0,0.150935,2.166667,1560.0,65.0
3,5.0,5.0,1.0,0.01,0.10,0.10,65.0,0.150935,2.166667,1560.0,65.0
4,5.0,5.0,1.0,0.10,0.01,0.01,65.0,0.150935,2.166667,1560.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...
12163,195.0,195.0,1.0,0.01,0.10,0.10,737.0,1.711366,24.566667,17688.0,737.0
12164,195.0,195.0,1.0,0.10,0.01,0.01,737.0,1.711366,24.566667,17688.0,737.0
12165,195.0,195.0,1.0,0.10,0.01,0.10,737.0,1.711366,24.566667,17688.0,737.0
12166,195.0,195.0,1.0,0.10,0.10,0.01,737.0,1.711366,24.566667,17688.0,737.0


In [27]:
df.to_csv("Simulation_1_24Hour.csv")

In [34]:
# keeping Beta 1 to Beta 4 the same, and C_Value the same 
# we can see how the B_Value will change the effect
# the user spamming the system at random hours 

df = pd.DataFrame(columns = ['B_VALUE', 'C_VALUE', 'BETA1', 'BETA2', 'BETA3', 'BETA4', 'transactions', 'ETH', 'User_age','timings','hours', 'days'])

def sim1(b, c, b1):
    global B_VALUE, C_VALUE, BETA1, BETA2, BETA3, BETA4
    B_VALUE = b
    C_VALUE = c
    BETA1 = b1
    user_age = 0
    total_time_passed = 0.0
    user_x = 0
    user_rep = 0
    repList = []
    xList = []
    timings = []
#     Age_List = []
    while user_rep < 99:
        arbitrary_time = np.random.randint(1, 168)
        timings.append(arbitrary_time)
        total_time_passed += arbitrary_time
        user_age = total_time_passed/24/30 #divide by 24 hours, divide by 30days
#         Age_List.append(user_age)
        user_x = calculateX_Buyer(user_x, arbitrary_time, int(user_age))
        xList.append(user_x)
        user_rep = calculateRep_Buyer(user_x)
        repList.append(user_rep)
        
    return len(repList), (len(repList))*2322070000000000/(10**18), total_time_passed, total_time_passed/24, user_age, timings



#the user tries to inflate within a month and price 100wei, time between transactions is one hour
for b in range(5, 100, 5):
    for c in range(5, 100, 5):
        b1 = 1
        for b2 in [0.01, 0.1]:
            for b3 in [0.01, 0.1]:
                for b4 in [0.01, 0.1]:
                    transactions, ETH, hours, days, user_age, timings = sim1(b, c, b1)
                    df.loc[len(df)] = [b, c, b1, b2, b3, b4, transactions, ETH, user_age,str(timings), hours, days]
    
df

,B_VALUE,C_VALUE,BETA1,BETA2,BETA3,BETA4,transactions,ETH,User_age,timings,hours,days
0,5,5,1,0.01,0.01,0.01,17,0.039475,2.366667,"[77, 93, 102, 36, 152, 144, 155, 111, 82, 113,...",1704.0,71.000000
1,5,5,1,0.01,0.01,0.10,21,0.048763,2.229167,"[44, 135, 118, 88, 79, 93, 78, 111, 15, 78, 54...",1605.0,66.875000
2,5,5,1,0.01,0.10,0.01,20,0.046441,2.402778,"[64, 128, 35, 31, 8, 38, 54, 130, 78, 146, 143...",1730.0,72.083333
3,5,5,1,0.01,0.10,0.10,21,0.048763,2.363889,"[40, 86, 71, 134, 61, 25, 9, 147, 65, 149, 4, ...",1702.0,70.916667
4,5,5,1,0.10,0.01,0.01,16,0.037153,2.216667,"[55, 122, 134, 153, 116, 115, 91, 116, 132, 23...",1596.0,66.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
2883,95,95,1,0.01,0.10,0.10,160,0.371531,17.919444,"[65, 142, 118, 26, 139, 101, 1, 53, 117, 81, 1...",12902.0,537.583333
2884,95,95,1,0.10,0.01,0.01,164,0.380819,18.073611,"[32, 9, 79, 101, 68, 166, 14, 111, 14, 109, 13...",13013.0,542.208333
2885,95,95,1,0.10,0.01,0.10,149,0.345988,18.205556,"[163, 119, 162, 62, 98, 109, 141, 113, 89, 81,...",13108.0,546.166667
2886,95,95,1,0.10,0.10,0.01,163,0.378497,17.947222,"[38, 13, 51, 31, 9, 111, 16, 60, 115, 48, 94, ...",12922.0,538.416667


In [35]:
df.to_csv("Simulation_1_Arbitrary_Time.csv")

# Simulation 2: Can a user create multiple different accounts, to influence reviews? (spam attack with un trusted users)

**Considerations**
1. How tolerant is the system?
2. How many accounts is needed before you can significantly increase?
3. What parameters can we set?


In [ ]:
# For the buyers we set 
A_VALUE = 50 # remains the same
B_VALUE = 5
C_VALUE = 95

A_VALUE_S = 50 # remains the same
B_VALUE_S = 5
C_VALUE_S = 3000


BETA1 = 1
BETA2 = 0.01
BETA3 = 0.01
BETA4 = 0.01
BETA_S = 0.001

# Simulation 3: Can a user create multiple different accounts, to influence reviews? (spam attack with fully trusted users)

**Considerations**
1. How tolerant is the system?
2. How many accounts is needed before you can significantly increase?
3. What parameters can we set?

# Simulation 4: Can a user create multiple different accounts, to influence reviews? (spam attack with mixed time gaps with low reputation score)

**Considerations**
1. How tolerant is the system?
2. How many accounts is needed before you can significantly increase?
3. What parameters can we set?

# Simulation 5: Can a user create multiple different accounts, to influence reviews? (spam attack with only long time durations between the attacks)

**Considerations**
1. How tolerant is the system?
2. How many accounts is needed before you can significantly increase?
3. What parameters can we set?